<a href="https://colab.research.google.com/github/gowrilvinod/Movie-review/blob/main/Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing dependencies


In [ ]:
import nltk
import numpy as np
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
nltk.download('punkt')
nltk.download('movie_reviews')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

Loading **data**

In [ ]:
positive_reviews = []
for fileid in movie_reviews.fileids('pos'):
    words = movie_reviews.words(fileid)
    positive_reviews.append(' '.join(words))

negative_reviews = []
for fileid in movie_reviews.fileids('neg'):
    words = movie_reviews.words(fileid)
    negative_reviews.append(' '.join(words))

Combining positive and negative reviews

In [ ]:
reviews = positive_reviews + negative_reviews
sentiments = np.concatenate([np.ones(len(positive_reviews)), np.zeros(len(negative_reviews))])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
maxlen = 100
X = pad_sequences(sequences, maxlen=maxlen)

Spliting data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, sentiments, test_size=0.2, random_state=42)

Building LSTM model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=maxlen))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


Compiling model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Training model

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
50/50 [==============================] - 15s 214ms/step - loss: 0.6907 - accuracy: 0.5375 - val_loss: 0.6733 - val_accuracy: 0.5625
Epoch 2/5
50/50 [==============================] - 11s 224ms/step - loss: 0.4869 - accuracy: 0.7950 - val_loss: 0.5663 - val_accuracy: 0.7225
Epoch 3/5
50/50 [==============================] - 12s 237ms/step - loss: 0.1358 - accuracy: 0.9588 - val_loss: 0.7104 - val_accuracy: 0.6825
Epoch 4/5
50/50 [==============================] - 12s 232ms/step - loss: 0.0592 - accuracy: 0.9856 - val_loss: 0.7736 - val_accuracy: 0.7350
Epoch 5/5
50/50 [==============================] - 11s 225ms/step - loss: 0.0201 - accuracy: 0.9962 - val_loss: 0.7439 - val_accuracy: 0.6925


Testing model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

13/13 [==============================] - 1s 62ms/step - loss: 0.7439 - accuracy: 0.6925
Test Accuracy: 0.6924999952316284


Predicting sentiment

In [ ]:
test_review = input("Enter the movie review: ")
test_sequence = tokenizer.texts_to_sequences([test_review])
test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
prediction = model.predict(test_sequence)[0][0]

if prediction >= 0.5:
    print("Sentiment: Positive")
else:
    print("Sentiment: Negative")

Enter the movie review: Average film
1/1 [==============================] - 0s 32ms/step
Sentiment: Negative
